In [4]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chains import RetrievalQA
from qdrant_client import QdrantClient

In [11]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# Load qdrant collection


In [5]:

# Khởi tạo Qdrant client
qdrant_client = QdrantClient(url="http://localhost:6333", prefer_grpc=True)

# Khởi tạo embeddings (có thể cần nếu dùng để search)
embeddings = OpenAIEmbeddings()

# Load collection từ Qdrant
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name="documents_tesla_q3",
    embeddings=embeddings,  # Cần để tìm kiếm vector
)

# Tạo retriever từ vectorstore
retriever = vectorstore.as_retriever()

/tmp/ipykernel_414491/2653360078.py:10: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


# web search

In [6]:
# define web search
web_search_tool = TavilySearchResults(k=5)

# Agentic RAG

In [7]:
# load llm
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.,  # Điều chỉnh độ sáng tạo của model
    max_tokens=512,   # Giới hạn số token đầu ra        
)

In [8]:
# define vector search
def vector_search(query: str):
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain.run(query)

In [9]:
vector_search("What was Tesla's total revenue in Q3?")

/tmp/ipykernel_414491/1923796800.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return qa_chain.run(query)


"Tesla's total revenue in Q3 was $25.2 billion."

In [10]:
# define web search
def web_search(query: str):
   return web_search_tool.run(query)

In [ ]:
from langchain_core.tools import tool


# Define tools
@tool
def vector_search(query: str) -> str:
    """Tool for searching the vector store.

    Args:
        query: input query of user
    """
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain.run(query)



@tool
def web_search(query: str) -> str:
    """Tool for performing web search.

    Args:
        query: input query of user
    """
    return web_search_tool.run(query)